# PyTorch Tutorial on Neural Networks

## Working with data
This Jupyter Notebook follows the PyTorch Tutorial using the FashinMNIST dataset. The link is available [here](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html).

There are text, computational vision and audio datasets available (OpenSource) in the TorchText, TorchVision and TorchAudio libraries.

In [1]:
# Adding imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Downloading training data from built-in open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data 
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

The ```Dataset``` is passed as an argument to ```DataLoader``` and wrapped as an iterable. This way it supports batching, sampling, shuffling and data loading. Each element in the iterable returns a batch of a given number of features and labels. Here, the batch size is of 64 elements.

In [3]:
batch_size = 64

# Create data loaders for training and testing
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

To define a Neural Network in PyTorch, we create a class that inherits from nn.Module. The layers are defined in ```__init__``` function. The ```forward``` function describes how data will pass through the network. 

It moves the operation to the GPU or MPS, if available, to accelerate the operations.

In [4]:
# Choose between cpu, gpu or mps device for training
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters
To train a model, we need a loss function and an optimizer.
- A ***loss function*** is a function that quantifies the difference between the predicted outputs of an algorithm and the actual target values. Here we have the [*Cross Entropy loss function*](https://www.datacamp.com/tutorial/the-cross-entropy-loss-function-in-machine-learning)
- An ***optimizer*** defines how the parameters are adapted to achieve the expected target value using the loss in the loss function. Here it is the [*Stochastic Gradient Descent* (SGD)](https://www.geeksforgeeks.org/ml-stochastic-gradient-descent-sgd/)

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Now, in a single loop, the model is fed in batches and make predictions on the training dataset. It backpropagates the prediction error from the loss function to adjust the model's parameter using the optimizer.

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Computes prediction error with the loss function
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}] {size:>5d}")

In [8]:
# Check the performance against the test dataset to evaluate learning
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, \
          Avg loss: {test_loss:>8f}\n")

The training process occurs over a number of iterations called *epochs*. During each iteration/epoch, the model learns parameters to make more accurate predictions. Here, the accuracy and loss at each epoch is printed. The goal is to see the increasing of accuracy and decrease of loss in each epoch.

In [9]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------------
Loss: 2.302674 [   64] 60000


Loss: 2.289059 [ 6464] 60000
Loss: 2.278941 [12864] 60000
Loss: 2.277173 [19264] 60000
Loss: 2.253172 [25664] 60000
Loss: 2.233164 [32064] 60000
Loss: 2.237074 [38464] 60000
Loss: 2.205402 [44864] 60000
Loss: 2.207637 [51264] 60000
Loss: 2.177548 [57664] 60000
Test Error: 
 Accuracy: 40.8%,           Avg loss: 2.168786

Epoch 2
-----------------------------
Loss: 2.178988 [   64] 60000
Loss: 2.166190 [ 6464] 60000
Loss: 2.118474 [12864] 60000
Loss: 2.134851 [19264] 60000
Loss: 2.085658 [25664] 60000
Loss: 2.029117 [32064] 60000
Loss: 2.056638 [38464] 60000
Loss: 1.981560 [44864] 60000
Loss: 1.989296 [51264] 60000
Loss: 1.920433 [57664] 60000
Test Error: 
 Accuracy: 53.5%,           Avg loss: 1.918233

Epoch 3
-----------------------------
Loss: 1.950707 [   64] 60000
Loss: 1.917555 [ 6464] 60000
Loss: 1.812820 [12864] 60000
Loss: 1.850494 [19264] 60000
Loss: 1.749069 [25664] 60000
Loss: 1.691694 [32064] 60000
Loss: 1.719075 [38464] 60000
Loss: 1.620263 [44864] 60000
Loss: 1.646767 [512

## Saving and Loading Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

## Making predictions
This model can now be used to make predictions.

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
miss_count = 0
with torch.no_grad():
    for i in range(len(test_data)):
        x, y = test_data[i][0], test_data[i][1]
        x = x.to(device)
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        if predicted != actual:
            miss_count += 1
            print(f"Predicted: {predicted}, Actual: {actual}")

print(f"Miss count: {miss_count}")

Predicted: Sandal, Actual: Sneaker
Predicted: Shirt, Actual: Coat
Predicted: T-shirt/top, Actual: Pullover
Predicted: Sneaker, Actual: Sandal
Predicted: Sneaker, Actual: Ankle boot
Predicted: Pullover, Actual: Coat
Predicted: Dress, Actual: T-shirt/top
Predicted: Coat, Actual: Dress
Predicted: T-shirt/top, Actual: Shirt
Predicted: Ankle boot, Actual: Sneaker
Predicted: Coat, Actual: Pullover
Predicted: Shirt, Actual: Pullover
Predicted: Pullover, Actual: Coat
Predicted: Shirt, Actual: Bag
Predicted: Dress, Actual: Pullover
Predicted: Coat, Actual: Dress
Predicted: Sneaker, Actual: Ankle boot
Predicted: Shirt, Actual: Pullover
Predicted: Shirt, Actual: Coat
Predicted: Coat, Actual: Shirt
Predicted: Coat, Actual: Shirt
Predicted: Shirt, Actual: Pullover
Predicted: Dress, Actual: Shirt
Predicted: Shirt, Actual: Coat
Predicted: Pullover, Actual: Coat
Predicted: Dress, Actual: Coat
Predicted: Bag, Actual: Shirt
Predicted: Pullover, Actual: Shirt
Predicted: Coat, Actual: Pullover
Predicted: 

Predicted: Coat, Actual: Pullover
Predicted: Trouser, Actual: T-shirt/top
Predicted: Shirt, Actual: Pullover
Predicted: Shirt, Actual: T-shirt/top
Predicted: Trouser, Actual: Dress
Predicted: Dress, Actual: T-shirt/top
Predicted: Dress, Actual: T-shirt/top
Predicted: Ankle boot, Actual: Sandal
Predicted: Sneaker, Actual: Sandal
Predicted: Pullover, Actual: Coat
Predicted: Pullover, Actual: Shirt
Predicted: Coat, Actual: Shirt
Predicted: Shirt, Actual: Coat
Predicted: Sandal, Actual: Sneaker
Predicted: Shirt, Actual: T-shirt/top
Predicted: Bag, Actual: T-shirt/top
Predicted: Shirt, Actual: Coat
Predicted: Pullover, Actual: Shirt
Predicted: Ankle boot, Actual: Sandal
Predicted: Sneaker, Actual: Sandal
Predicted: Ankle boot, Actual: Sandal
Predicted: Pullover, Actual: Dress
Predicted: Shirt, Actual: T-shirt/top
Predicted: T-shirt/top, Actual: Shirt
Predicted: T-shirt/top, Actual: Shirt
Predicted: Dress, Actual: Shirt
Predicted: Coat, Actual: Pullover
Predicted: T-shirt/top, Actual: Shirt
